In [1]:
!pip install stable_baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 11.2 MB/s 
     |████████████████████████████████| 1.5 MB 49.2 MB/s 
     |████████████████████████████████| 1.6 MB 10.0 MB/s 
     |████████████████████████████████| 237 kB 50.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51 kB 1.6 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616822 sha256=ecec57689eeb6a02eb2372e15f1598ff3c8bf0ccacb24357351242fc962a1292
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441028 sha256=efcbd8fe7d0690b0bbbcea8ff6ff4f072aa3988bd05b5dc169338a08c1a9dc40
  Stored in directory: /root/.cache/pip/wh

In [2]:
import tensorflow as tf 
import numpy as np
import time
from PIL import Image
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C

In [ ]:
env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=4, seed=0)
# Frame-stacking with 4 frames
env = VecFrameStack(env, n_stack=4)

model = A2C("CnnPolicy", env, verbose=1)
#model = A2C.load("a2a_model350", env=env)
model.learn(total_timesteps=250_000) 
model.save("a2a_model600")

obs = env.reset()
max_reward = 17
rewards_cum = [0,0,0,0]
best_counter = 0
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)    
    image = env.render("rgb_array")

    rewards_cum[0] += rewards[0]
    rewards_cum[1] += rewards[1]
    rewards_cum[2] += rewards[2]
    rewards_cum[3] += rewards[3]

    for index in range(4):
        if info[index]["lives"] == 0:
            rewards_cum[index] = 0

    if max(rewards_cum) > max_reward:
        print(rewards)
        i = Image.fromarray(image)
        i.save(f"best_sample{best_counter}.png")
        best_counter+=1